# Collaborative Filtlering

This is an updated recommender system using Collaborative filtering KNN on 'Book-Crossing Dataset'

In [39]:
#import liabraries
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import sklearn
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
import warnings

In [40]:
# upload and view dataset 
book_df = pd.read_csv("D:\\DS_portfolio\\Book_recommender\\BX-Books.csv",sep=';', error_bad_lines=False, encoding="latin-1")
book_df.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlS', 'imageUrlM', 'imageUrlL']
user_df = pd.read_csv('D:\\DS_portfolio\\Book_recommender\\BX-Users.csv', sep=';', error_bad_lines=False, encoding ="latin-1")
user_df.columns=['User-ID', 'Location', 'Age']
rating_df = pd.read_csv('D:\\DS_portfolio\\Book_recommender\\BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding ="latin-1")
rating_df.columns = ['User-ID', 'ISBN', 'Book-Rating']

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\Users\singh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.p

# Book data

In [3]:
book_df.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
#drop urls
book_df.drop(['imageUrlS', 'imageUrlL', 'imageUrlM'],axis=1,inplace=True)

In [5]:
book_df.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


# User Data

In [6]:
user_df.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


# Rating Data

In [7]:
rating_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


# For the statistical significance we will look into the popular books, books with 100 or more ratings

We merge book and rating data, keep only rating, userid, isbn and book title columns

In [8]:
book_rating = pd.merge(book_df,rating_df, on ='ISBN')
columns = ['bookAuthor','yearOfPublication','publisher']
book_rating = book_rating.drop(columns, axis=1)
book_rating.head()

,ISBN,bookTitle,User-ID,Book-Rating
0,0195153448,Classical Mythology,2,0
1,0002005018,Clara Callan,8,5
2,0002005018,Clara Callan,11400,0
3,0002005018,Clara Callan,11676,8
4,0002005018,Clara Callan,41385,0


we now group by book titles and create a new column for total rating count.

In [9]:
book_rating = book_rating.dropna(axis=0, subset=['bookTitle'])

In [10]:
rating_count = (book_rating.groupby('bookTitle')['Book-Rating'].count().reset_index().rename(columns = {'Book-Rating':'total_rating_count'})[['bookTitle','total_rating_count']])
rating_count.head()

,bookTitle,total_rating_count
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


We will combine rating data with total rating data to filter most rated books

In [14]:
combined_rating = book_rating.merge(rating_count, left_on='bookTitle', right_on='bookTitle', how='left')
print(combined_rating)

               ISBN                                          bookTitle  \
0        0195153448                                Classical Mythology   
1        0002005018                                       Clara Callan   
2        0002005018                                       Clara Callan   
3        0002005018                                       Clara Callan   
4        0002005018                                       Clara Callan   
...             ...                                                ...   
1031131  0440400988                         There's a Bat in Bunk Five   
1031132  0525447644                            From One to One Hundred   
1031133  006008667X  Lily Dale : The True Story of the Town that Ta...   
1031134  0192126040                        Republic (World's Classics)   
1031135  0767409752  A Guided Tour of Rene Descartes' Meditations o...   

         User-ID  Book-Rating  total_rating_count  
0              2            0                   2  
1      

explore the statistics of total_rating_count

In [15]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(rating_count['total_rating_count'].describe())

count   241071.000
mean         4.277
std         16.739
min          1.000
25%          1.000
50%          1.000
75%          3.000
max       2502.000
Name: total_rating_count, dtype: float64


In [16]:
print(rating_count['total_rating_count'].quantile(np.arange(.9, 1, .01)))

0.900    7.000
0.910    8.000
0.920    9.000
0.930   10.000
0.940   11.000
0.950   13.000
0.960   16.000
0.970   20.000
0.980   29.000
0.990   50.000
Name: total_rating_count, dtype: float64


rating_threshold =100

In [20]:
rating_threshold = 100
most_rated_book = combined_rating.query('total_rating_count >= @rating_threshold')
print(most_rated_book.head())

          ISBN               bookTitle  User-ID  Book-Rating  \
30  0399135782  The Kitchen God's Wife        8            0   
31  0399135782  The Kitchen God's Wife    11676            9   
32  0399135782  The Kitchen God's Wife    29526            9   
33  0399135782  The Kitchen God's Wife    36836            0   
34  0399135782  The Kitchen God's Wife    46398            9   

    total_rating_count  
30                 311  
31                 311  
32                 311  
33                 311  
34                 311  


# Filtering user data on location base

In [26]:
combined = most_rated_book.merge(user_df, left_on='User-ID', right_on='User-ID', how='left')

nl_user_rating = combined[combined['Location'].str.contains("netherlands")]
nl_user_rating = nl_user_rating.drop('Age', axis=1)
print(nl_user_rating.head())

           ISBN                             bookTitle  User-ID  Book-Rating  \
33   0440234743                         The Testament   277478            0   
167  0440234743                         The Testament    94074            0   
331  0440234743                         The Testament   199092            3   
536  0452264464  Beloved (Plume Contemporary Fiction)   116210            0   
564  0452264464  Beloved (Plume Contemporary Fiction)   157799            5   

     total_rating_count                                Location  
33                  617     schiedam, zuid-holland, netherlands  
167                 617       zevenaar, gelderland, netherlands  
331                 617    rotterdam, zuid-holland, netherlands  
536                 180  oisterwijk, noord-brabant, netherlands  
564                 180   heemstede, noord holland, netherlands  


# KNN implementation

we convert the table into 2D matrix and fill the missing values with zero

In [27]:
nl_user_rating = nl_user_rating.drop_duplicates(['User-ID', 'bookTitle'])
nl_user_rating_pivot = nl_user_rating.pivot(index='bookTitle', columns='User-ID',
                                                          values='Book-Rating').fillna(0)
nl_user_rating_matrix = csr_matrix(nl_user_rating_pivot.values)

# Finding the nearest neighbors

In [30]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(nl_user_rating_matrix)
NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)


NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

# Test the model and make recommendations

In [32]:
query_index = np.random.choice(nl_user_rating_pivot.shape[0])
distances, indices = model_knn.kneighbors(nl_user_rating_pivot.iloc[query_index, :].values.reshape(1, -1),
                                          n_neighbors=6)
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(nl_user_rating_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, nl_user_rating_pivot.index[indices.flatten()[i]],
                                                       distances.flatten()[i]))



Recommendations for The Pilot's Wife : A Novel:

1: The Best Laid Plans, with distance of 1.0:
2: The Amazing Adventures of Kavalier &amp; Clay, with distance of 1.0:
3: The Amber Spyglass (His Dark Materials, Book 3), with distance of 1.0:
4: The Alchemist: A Fable About Following Your Dream, with distance of 1.0:
5: The Bean Trees, with distance of 1.0:


# Matrix factorization

Now we will use matrix factorization technique of collaborative filtering.
The Matrix Factorization techniques are usually more effective, because they allow users to discover the latent (hidden)features underlying the interactions between users and items (books).

# Singular Value Decomposition Model

SVD is one of the matrix factorization model we will use here. Similar to KNN we wil make a pivot table and fill the missing values with zero

In [33]:
nl_user_rating_pivot2 = nl_user_rating.pivot(index='User-ID', columns='bookTitle',
                                                           values='Book-Rating').fillna(0)
print(nl_user_rating_pivot2.head())



bookTitle  24 Hours  84 Charing Cross Road  A Case of Need  \
User-ID                                                      
695           0.000                  0.000           0.000   
2359          0.000                  0.000           0.000   
5333          0.000                  0.000           0.000   
7856          0.000                  0.000           0.000   
10681         0.000                  0.000           0.000   

bookTitle  A Child Called \It\": One Child's Courage to Survive"  \
User-ID                                                            
695                                                    0.000       
2359                                                   0.000       
5333                                                   0.000       
7856                                                   0.000       
10681                                                  0.000       

bookTitle  A Darkness More Than Night  \
User-ID                                 
695   

In [34]:
nl_user_rating_pivot2.shape

(196, 375)

In [36]:
X = nl_user_rating_pivot2.values.T
X.shape

(375, 196)

We then transpose this utility matrix, so that the bookTitles become rows and userIDs become columns.
After using TruncatedSVD to decompose it, we fit it into the model for dimensionality reduction.

In [37]:
SVD = TruncatedSVD(n_components=12, random_state=17)
matrix = SVD.fit_transform(X)
matrix.shape

(375, 12)

We calculate the Pearson’s R correlation coefficient for every book pair in our final matrix.

In [41]:
warnings.filterwarnings("ignore", category=RuntimeWarning)
corr = np.corrcoef(matrix)

To compare this with the results from kNN, we pick the book “The Alchemist: A Fable About Following Your Dream” to find the books that have high correlation coefficients (between 0.9 and 1.0) with it.


In [42]:
book_title = nl_user_rating_pivot2.columns
book_list = list(book_title)

coffey_hands = book_list.index("The Alchemist: A Fable About Following Your Dream")
print(coffey_hands)

247


In [43]:
corr_coffey_hands = corr[coffey_hands]

print(list(book_title[(corr_coffey_hands <= 0.9)]))

['24 Hours', '84 Charing Cross Road', 'A Case of Need', 'A Child Called \\It\\": One Child\'s Courage to Survive"', 'A Man in Full', 'A Map of the World', 'A Painted House', 'A Prayer for Owen Meany', "A Thousand Acres (Ballantine Reader's Circle)", 'A Time to Kill', 'A Walk in the Woods: Rediscovering America on the Appalachian Trail (Official Guides to the Appalachian Trail)', 'A Walk to Remember', 'A Widow for One Year', 'Abduction', 'About a Boy', 'About a Boy (Movie Tie-In)', 'Absolute Power', 'Acceptable Risk', 'Alias Grace', 'All That Remains (Kay Scarpetta Mysteries (Paperback))', 'American Gods', 'American Psycho (Vintage Contemporaries)', 'An Instance of the Fingerpost', "Angela's Ashes (MMP) : A Memoir", 'Angelas Ashes', 'Angels &amp; Demons', 'Basket Case', 'Bastard Out of Carolina', 'Beloved', 'Beloved (Plume Contemporary Fiction)', 'Breathing Lessons', "Bridget Jones's Diary", 'C Is for Corpse (Kinsey Millhone Mysteries (Paperback))', 'Carnal Innocence', 'Catch Me If You 